In [2]:
# Install if needed
# !pip install streamlit scikit-learn matplotlib seaborn

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import classification_report
import pickle

# -----------------------------
# 1. Train model
# -----------------------------
csv_file = "main data.csv"   # 👈 your dataset
target_col = "state"         # 👈 target column

df = pd.read_csv(csv_file)
X = df.drop(columns=[target_col])
y = df[target_col]

numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()

# scaler choice
scaler_choice = "standard"   # "standard" or "minmax"
scaler = StandardScaler() if scaler_choice == "standard" else MinMaxScaler()

preprocess = ColumnTransformer([
    ("num", Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", scaler)
    ]), numeric_cols)
], remainder="drop")

rf = RandomForestClassifier(n_estimators=300, random_state=42)
pipe = Pipeline([
    ("preprocess", preprocess),
    ("model", rf)
])
pipe.fit(X, y)

# Select top 10 features
importances = pipe.named_steps["model"].feature_importances_
top_features = [numeric_cols[i] for i in np.argsort(importances)[-10:]]

# Final model with top features
X_train, X_test, y_train, y_test = train_test_split(
    X[top_features], y, test_size=0.2, random_state=42, stratify=y
)

preprocess_final = ColumnTransformer([
    ("num", Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", scaler)
    ]), top_features)
])

pipe_final = Pipeline([
    ("preprocess", preprocess_final),
    ("model", RandomForestClassifier(n_estimators=300, random_state=42))
])
pipe_final.fit(X_train, y_train)

with open("model.pkl", "wb") as f:
    pickle.dump((pipe_final, top_features, X_test, y_test, scaler_choice), f)

print("✅ Model trained. Top features:", top_features)

# -----------------------------
# 2. Create Streamlit app
# -----------------------------
app_code = f"""
import streamlit as st
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import classification_report, roc_curve, auc
import matplotlib.pyplot as plt

st.set_page_config(page_title="Cancer Prediction Prototype", layout="wide")

# ---- Custom CSS ----
st.markdown(
    \"\"\"
    <style>
    body, .stApp {{
        font-family: 'Arial', sans-serif;
    }}
    h1, h2, h3, h4, h5, h6 {{
        font-family: 'Arial Black', sans-serif;
    }}
    .stButton>button {{
        font-size: 16px;
        font-weight: bold;
    }}
    </style>
    \"\"\",
    unsafe_allow_html=True
)

# ---- Logos + Teams ----
st.markdown("## Partner Research Groups")


col1, col2, _ = st.columns([1, 1, 6])

with col1:
    st.image("mllogo.JPG", width=150)
    st.markdown("[**ML-Healthcare Group**](https://mlhealthcare.framer.website/)")

with col2:
    st.image("logo_TP.png", width=150)
    st.markdown("[**SpentaGen Group**](https://spentagen.com/)")
    

st.title("Cancer Prediction Prototype")
st.caption("⚠️ This tool is a research prototype only. It does not replace medical diagnosis or consultation.")

# ---- Load model ----
with open("model.pkl", "rb") as f:
    model, top_features, X_test, y_test, scaler_choice = pickle.load(f)

st.subheader("Enter values for the 10 selected features (raw input; normalized internally with " + scaler_choice + ")")
inputs = {{}}
cols = st.columns(2)
for i, f in enumerate(top_features):
    with cols[i % 2]:
        val = st.number_input(f, value=float(X_test[f].mean()))
        inputs[f] = val

if st.button("Run Prediction"):
    user_df = pd.DataFrame([inputs])
    proba = model.predict_proba(user_df)[0,1]
    pred = int(proba >= 0.5)

    if pred == 1:
        st.error(f"🔴 Model result (prototype): High cancer risk (probability = {{proba:.3f}})")
    else:
        st.success(f"🟢 Model result (prototype): Low cancer risk (probability = {{1-proba:.3f}})")

    st.info("⚠️ This result is experimental. Please consult a physician for further evaluation.")

    # Classification report
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    st.subheader("Classification Report")
    st.dataframe(pd.DataFrame(report).transpose())

    # ROC curve
    st.subheader("ROC Curve")
    fpr, tpr, _ = roc_curve(y_test, model.predict_proba(X_test)[:,1])
    roc_auc = auc(fpr, tpr)
    fig, ax = plt.subplots()
    ax.plot(fpr, tpr, label=f"AUC = {{roc_auc:.3f}}")
    ax.plot([0,1],[0,1],"--", color="gray")
    ax.set_xlabel("False Positive Rate")
    ax.set_ylabel("True Positive Rate")
    ax.legend()
    st.pyplot(fig)

# ---- References ----
st.markdown("## References")
st.markdown(\"\"\" 
- [WHFDL: an explainable method based on World Hyper-heuristic and Fuzzy Deep Learning approaches for gastric cancer detection using metabolomics data](https://biodatamining.biomedcentral.com/articles/10.1186/s13040-025-00486-1)  
- [Metabolomic machine learning predictor for diagnosis and prognosis of gastric cancer](https://www.nature.com/articles/s41467-024-46043-y)
\"\"\")
"""

with open("app.py", "w", encoding="utf-8") as f:
    f.write(app_code)

print("✅ app.py created. Run it with:  streamlit run app.py")


# -----------------------------
# 3. Run Streamlit
# -----------------------------
# !streamlit run app.py --server.headless true --server.port 8501

# OR


# run cmd ---->
# windows + R ----> type "cmd" and enter,

# change path:
# cd %USERPROFILE%\Desktop\GTM NEW

# run the app by streamlit ---->
# streamlit run app.py


✅ Model trained. Top features: ['Glycerophosphorylcholine_pos-080', 'Pyroglutamic acid_neg-072', 'GSSG_pos-086', 'Ribose 5-phosphate_neg-074', 'Lactic acid_neg-055', '4-Hydroxyphenylpyruvic acid_neg-013', 'Uracil_pos-156', 'Glycerate-2P_Glycerate-3P_neg-006', 'Succinate_neg-079', 'Uridine_neg-088']
✅ app.py created. Run it with:  streamlit run app.py
